In [1]:
## TFM_01_scrapping_data.ipynb

In [2]:
import datetime
import timedelta
import os
from typing import List, Dict, Tuple, Any, Union
import pandas as pd
import asyncio
import aiohttp
import pytz
from IPython.display import display

In [ ]:
URL_BASE_BINANCE = "https://api.binance.com/api/v3"
DIRECTORIO_SALIDA = 'binance_data' 

In [4]:
def funcion_definir_ayer():

    zona_utc = pytz.UTC
    hoy_utc = datetime.datetime.now(zona_utc).replace(hour=0, minute=0, second=0, microsecond=0)
    inicio_dia_ayer = hoy_utc - datetime.timedelta(days=1)
    fin_dia_ayer = hoy_utc - datetime.timedelta(microseconds=1)
    
    return inicio_dia_ayer, fin_dia_ayer

In [5]:
def funcion_calcular_fechas(fecha_texto: str):
    """Calcula fechas de inicio y fin del día."""
    if fecha_texto:
        try:
            fecha_entrada = datetime.datetime.strptime(fecha_texto, '%d/%m/%Y')
            inicio_dia = fecha_entrada.replace(hour=0, minute=0, second=0, microsecond=0)
            fin_dia = fecha_entrada.replace(hour=23, minute=59, second=59, microsecond=999999)
            zona_utc = pytz.UTC
            inicio_dia_utc = zona_utc.localize(inicio_dia)
            fin_dia_utc = zona_utc.localize(fin_dia)
            print(f"Fecha: {inicio_dia_utc.strftime('%Y-%m-%d')}")
            return inicio_dia_utc, fin_dia_utc
        except ValueError:
            print("Formato inválido. Se usará ayer.")
            return funcion_definir_ayer()
    else:
        print("Sin fecha. Se usará ayer.")
        return funcion_definir_ayer()

In [6]:
async def funcion_obtener_velas_simbolo(sesion: aiohttp.ClientSession, simbolo: str, inicio_dia_utc: datetime.datetime, fin_dia_utc: datetime.datetime):
    """Obtiene velas de Binance para un símbolo y rango de fechas."""
    punto_final = f"{URL_BASE_BINANCE}/klines"
    mitad_dia_utc = inicio_dia_utc + datetime.timedelta(hours=12)

    inicio_ms_1 = int(inicio_dia_utc.timestamp() * 1000)
    fin_ms_1 = int(mitad_dia_utc.timestamp() * 1000) - 1

    inicio_ms_2 = int(mitad_dia_utc.timestamp() * 1000)
    fin_ms_2 = int(fin_dia_utc.timestamp() * 1000)

    todas_velas = []
    simbolo_par = f'{simbolo}USDT'

    parametros_1 = {'symbol': simbolo_par, 'interval': '1m', 'startTime': inicio_ms_1, 'endTime': fin_ms_1, 'limit': 1000}
    parametros_2 = {'symbol': simbolo_par, 'interval': '1m', 'startTime': inicio_ms_2, 'endTime': fin_ms_2, 'limit': 1000}

    try:
        print(f"  Obteniendo primera mitad para {simbolo}...")
        async with sesion.get(punto_final, params=parametros_1) as respuesta_1:
            if respuesta_1.status == 429:
                reintentar_despues = int(respuesta_1.headers.get('Retry-After', 5))
                print(f"  Límite alcanzado {simbolo} (1/2), esperando {reintentar_despues}s")
                await asyncio.sleep(reintentar_despues)
                return await funcion_obtener_velas_simbolo(sesion, simbolo, inicio_dia_utc, fin_dia_utc)

            if respuesta_1.status == 200:
                primera_mitad = await respuesta_1.json()
                todas_velas.extend(primera_mitad)
                print(f"  Primera mitad {simbolo}: {len(primera_mitad)} minutos")
            else:
                error_texto = await respuesta_1.text()
                print(f"  Error en la  primera mitad {simbolo} {respuesta_1.status}, {error_texto}")
                return [], f"Error API {respuesta_1.status} (1/2): {error_texto[:100]}"

        await asyncio.sleep(1)

        print(f"  Obteniendo segunda mitad para {simbolo}...")
        async with sesion.get(punto_final, params=parametros_2) as respuesta_2:
            if respuesta_2.status == 429:
                reintentar_despues = int(respuesta_2.headers.get('Retry-After', 5))
                print(f"  Límite alcanzado {simbolo} (2/2), esperando {reintentar_despues}s")
                await asyncio.sleep(reintentar_despues)
                async with sesion.get(punto_final, params=parametros_2) as respuesta_2_reintento:
                    if respuesta_2_reintento.status == 200:
                        segunda_mitad = await respuesta_2_reintento.json()
                        todas_velas.extend(segunda_mitad)
                        print(f"  Segunda mitad {simbolo} (reintento): {len(segunda_mitad)} minutos")
                    else:
                         error_texto = await respuesta_2_reintento.text()
                         print(f" Error en la segunda mitad {simbolo} (reintento) {respuesta_2_reintento.status}, {error_texto}")

            elif respuesta_2.status == 200:
                segunda_mitad = await respuesta_2.json()
                todas_velas.extend(segunda_mitad)
                print(f"  Segunda mitad {simbolo}: {len(segunda_mitad)} minutos")
            else:
                error_texto = await respuesta_2.text()
                print(f"  Error en la segunda mitad {simbolo} {respuesta_2.status}, {error_texto}")

        minutos_totales = len(todas_velas)
        if minutos_totales == 0 and respuesta_1.status != 200:

             return [], f"Error API {respuesta_1.status} (1/2)"
        elif minutos_totales < 1440:
            print(f"  Datos incompletos {simbolo}. Obtenidos {minutos_totales}/1440 minutos")
            return todas_velas, "Datos Incompletos" 
        else:
            print(f"  Datos completos {simbolo}: {minutos_totales} minutos")
            return todas_velas, "Completo"

    except aiohttp.ClientConnectorError as e:
        print(f"  Error de conexión {simbolo} {str(e)}")
        return [], f"Error de conexión: {str(e)}"
    except Exception as e:
        print(f"  Excepción {simbolo} {str(e)}")
        return [], f"Excepción: {str(e)}"

In [7]:
def funcion_procesar_velas(velas_crudas: List[Dict], simbolo: str):
    """Procesa velas crudas y retorna un DataFrame."""
    if not velas_crudas:
        return pd.DataFrame(), "Sin datos"

    columnas_df_inicial = [
        'timestamp_ms', 'open', 'high', 'low', 'close', 'volume',
        'close_time_ms', 'quote_asset_volume', 'number_of_trades',
        'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
    ]

    if velas_crudas and len(velas_crudas[0]) != len(columnas_df_inicial):
        print(f"  Columnas inesperadas en {simbolo}.")
        try:
            datos = pd.DataFrame(velas_crudas, columns=columnas_df_inicial)
        except ValueError:
            print(f"  Error al crear DataFrame para {simbolo}.")
            return pd.DataFrame(), "Error de formato"
    else:
        datos = pd.DataFrame(velas_crudas, columns=columnas_df_inicial)

    datos['timestamp'] = pd.to_datetime(datos['timestamp_ms'], unit='ms', utc=True)
    datos['symbol'] = simbolo

    columnas_finales = [
        'timestamp', 'symbol', 'open', 'high', 'low', 'close', 'volume',
        'quote_asset_volume', 'number_of_trades',
        'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume'
    ]
    columnas_a_seleccionar = [col for col in columnas_finales if col in datos.columns]
    datos = datos[columnas_a_seleccionar]

    columnas_numericas = [
        'open', 'high', 'low', 'close', 'volume', 'quote_asset_volume',
        'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume'
    ]
    for col in columnas_numericas:
        if col in datos.columns:
            datos[col] = pd.to_numeric(datos[col], errors='coerce')

    columnas_criticas_nan = ['open', 'high', 'low', 'close', 'volume']
    filas_antes = len(datos)
    subset_dropna = [col for col in columnas_criticas_nan if col in datos.columns]
    if subset_dropna:
        datos = datos.dropna(subset=subset_dropna)
    filas_despues = len(datos)
    if filas_antes > filas_despues:
        print(f"  Se eliminaron {filas_antes - filas_despues} filas nulas para {simbolo}.")

    if datos.empty:
        return pd.DataFrame(), "Vacío tras procesar"

    columnas_a_verificar_cero = [
        'quote_asset_volume', 'number_of_trades',
        'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume'
    ]
    columnas_con_ceros = []
    for col in columnas_a_verificar_cero:
        if col in datos.columns and (datos[col].fillna(0) == 0).all():
            columnas_con_ceros.append(col)

    if columnas_con_ceros:
        print(f"{simbolo} tiene columnas con solo ceros: {', '.join(columnas_con_ceros)}")
        return datos, f"Valores Cero ({', '.join(columnas_con_ceros)})"

    return datos, "Procesado OK"

In [8]:
async def funcion_obtener_procesar_lote(lote_simbolos: List[str], inicio_dia_utc: datetime.datetime, fin_dia_utc: datetime.datetime) -> Tuple[List[any], List[dict]]:
    """Obtiene, procesa y agrupa datos de símbolos financieros."""
    lista_dataframes_lote = []
    lista_fallos_lote = []

    async with aiohttp.ClientSession() as sesion:
        tareas = []
        for simbolo in lote_simbolos:
            await asyncio.sleep(0.1)
            tareas.append(asyncio.create_task(funcion_obtener_velas_simbolo(sesion, simbolo, inicio_dia_utc, fin_dia_utc)))

        resultados_crudos = await asyncio.gather(*tareas)

    fecha_str = inicio_dia_utc.strftime('%Y-%m-%d')

    for simbolo, (velas_simbolo, estado_obtencion) in zip(lote_simbolos, resultados_crudos):
        minutos_obtenidos = len(velas_simbolo)

        if not velas_simbolo and estado_obtencion not in ("Completo", "Datos Incompletos"):
            lista_fallos_lote.append({
                'simbolo': simbolo, 'fecha': fecha_str,
                'estado': estado_obtencion,
                'minutos_obtenidos': minutos_obtenidos
            })
            print(f"Omitiendo {simbolo}: {estado_obtencion}")
            continue

        datos_df, estado_procesado = funcion_procesar_velas(velas_simbolo, simbolo)

        if datos_df.empty:

            if not any(f['simbolo'] == simbolo for f in lista_fallos_lote):
                if estado_obtencion in ("Completo", "Datos Incompletos"):
                     lista_fallos_lote.append({
                        'simbolo': simbolo, 'fecha': fecha_str,
                        'estado': estado_procesado,
                        'minutos_obtenidos': minutos_obtenidos
                     })
                else:
                     lista_fallos_lote.append({
                        'simbolo': simbolo, 'fecha': fecha_str,
                        'estado': estado_obtencion,
                        'minutos_obtenidos': minutos_obtenidos
                     })
                print(f"Omitiendo {simbolo}: DataFrame vacío ({estado_procesado}). Obtención: {estado_obtencion}")
            continue

        lista_dataframes_lote.append(datos_df)
        print(f"Procesados {len(datos_df)} registros para {simbolo}")

        if estado_obtencion == "Datos Incompletos":
             lista_fallos_lote.append({
                 'simbolo': simbolo, 'fecha': fecha_str,
                 'estado': estado_obtencion,
                 'minutos_obtenidos': minutos_obtenidos
             })
        elif estado_procesado.startswith("Valores Cero"):
             lista_fallos_lote.append({
                 'simbolo': simbolo, 'fecha': fecha_str,
                 'estado': estado_procesado,
                 'minutos_obtenidos': len(datos_df)
             })

    return lista_dataframes_lote, lista_fallos_lote

In [9]:
def funcion_guardar_datos(lista_todos_dataframes: List[pd.DataFrame], inicio_dia_utc: datetime.datetime):
    """Guarda datos combinados de DataFrames en un archivo CSV."""
    if not lista_todos_dataframes:
        print("No hay DataFrames válidos.")
        return

    columnas_requeridas = ['timestamp', 'symbol']
    dataframes_validos = []
    for i, df in enumerate(lista_todos_dataframes):
        if all(col in df.columns for col in columnas_requeridas):
            dataframes_validos.append(df)
        else:
            print(f"DataFrame {i} omitido por faltar columnas.")

    if not dataframes_validos:
        print("No hay DataFrames válidos.")
        return

    datos_combinados = pd.concat(dataframes_validos, ignore_index=True)

    if datos_combinados.empty:
        print("DataFrame combinado vacío.")
        return

    fecha_archivo = inicio_dia_utc.strftime('%Y%m%d')
    nombre_archivo = f'datos_binance_minuto_{fecha_archivo}.csv'
    ruta_archivo_salida = os.path.join(DIRECTORIO_SALIDA, nombre_archivo)

    datos_combinados = datos_combinados.sort_values(by=['timestamp', 'symbol'])
    datos_combinados = datos_combinados.drop_duplicates(subset=['timestamp', 'symbol'], keep='last')

    try:
        datos_combinados.to_csv(ruta_archivo_salida, index=False, date_format='%Y-%m-%d %H:%M:%S%z')
        print(f"Datos guardados en {ruta_archivo_salida}. Filas: {len(datos_combinados)}")
    except Exception:
        print(f"Error al guardar el archivo CSV {ruta_archivo_salida}.")

In [10]:
def funcion_guardar_informe_fallos(lista_todos_fallos: List[Dict], inicio_dia_utc: datetime.datetime):
    """Guarda un informe de fallos en un archivo CSV."""
    if not lista_todos_fallos:
        print("No hubo fallos.")
        return

    datos_fallidos = pd.DataFrame(lista_todos_fallos)
    datos_fallidos = datos_fallidos.sort_values(by=['fecha', 'estado', 'simbolo'])
    datos_fallidos = datos_fallidos.drop_duplicates(subset=['simbolo', 'fecha', 'estado'], keep='last')

    fecha_archivo = inicio_dia_utc.strftime('%Y%m%d')
    nombre_archivo_fallos = f'informe_fallos_{fecha_archivo}.csv'
    ruta_archivo_fallos = os.path.join(DIRECTORIO_SALIDA, nombre_archivo_fallos)

    try:
        datos_fallidos.to_csv(ruta_archivo_fallos, index=False)
        print(f"Informe guardado en {ruta_archivo_fallos}")

        print("\nResumen de problemas:")
        print(f"Total: {len(datos_fallidos)}")
        if not datos_fallidos.empty:
            print("Distribución por estado:")
            print(datos_fallidos['estado'].value_counts().to_string())

    except Exception:
        print(f"Error al guardar el informe en {ruta_archivo_fallos}.")

In [11]:
async def proceso_principal():
    """Proceso principal para extraer y guardar datos."""
    fecha_entrada_usuario = input("Introduce la fecha (DD/MM/YYYY) o Intro para ayer: ").strip()

    inicio_dia_seleccionado, fin_dia_seleccionado = funcion_calcular_fechas(fecha_entrada_usuario)
    os.makedirs(DIRECTORIO_SALIDA, exist_ok=True)

    lista_simbolos_completa = [
        "1INCH", "AAVE", "ACA", "ACE", "ACH", "ACM", "ADA", "ADX", "AERGO", "AEVO",
        "AGLD", "AI", "AIDOGE", "AKRO", "ALCX", "ALGO", "ALICE", "ALPACA", "ALPHA",
        "ALPINE", "ALT", "AMB", "AMP", "ANKR", "APE", "API3", "APT", "AR", "ARB",
        "ARDR", "ARK", "ARKM", "ARPA", "ASR", "AST", "ASTR", "ATA", "ATH", "ATM",
        "ATOM", "AUCTION", "AUDIO", "AVA", "AVAX", "AXL", "AXS", "BABYDOGE", "BADGER",
        "BAKE", "BAL", "BANANA", "BAND", "BAR", "BAT", "BB", "BCH", "BEAM", "BEL",
        "BETA", "BICO", "BIFI", "BIGTIME", "BLOCK", "BLUR", "BLZ", "BNB", "BNT",
        "BNX", "BOME", "BONE", "BONK", "BRETT", "BSV", "BSW", "BTC", "BURGER", "C98",
        "CAKE", "CELO", "CELR", "CETUS", "CFX", "CHESS", "CHR", "CHZ", "CITY", "CKB",
        "CLV", "COMBO", "COMP", "COS", "COTI", "CRO", "CRV", "CTK", "CTSI", "CTXC",
        "CVC", "CVX", "CYBER", "DAI", "DAR", "DASH", "DATA", "DEGO", "DENT", "DEXE",
        "DF", "DGB", "DIA", "DODO", "DOGE", "DOGS", "DOT", "DUSK", "DYDX", "DYM",
        "EDU", "EGLD", "ELF", "ENA", "ENJ", "ENS", "EOS", "ERN", "ETC", "ETH",
        "ETHFI", "ETHW", "EUR", "FARM", "FET", "FIDA", "FIL", "FIO", "FIS", "FLM",
        "FLOKI", "FLOW", "FLR", "FLUX", "FORTH", "FOXY", "FTM", "FUN", "FXS", "G",
        "GALA", "GAS", "GFT", "GHST", "GLM", "GLMR", "GMT", "GMX", "GNO", "GNS",
        "GPT", "GRT", "GTC", "HARD", "HBAR", "HFT", "HIFI", "HIGH", "HIVE", "HOOK",
        "HOT", "ICP", "ICX", "ID", "IDEX", "ILV", "IMX", "INJ", "IO", "IOST", "IOTX",
        "IQ", "IRIS", "JASMY", "JOE", "JST", "JTO", "JUP", "JUV", "KAS", "KAVA",
        "KDA", "KEY", "KISHU", "KLAY", "KMD", "KNC", "KP3R", "KSM", "LAZIO", "LDO",
        "LEVER", "LINA", "LINK", "LISTA", "LIT", "LOKA", "LOOKS", "LPT", "LQTY",
        "LRC", "LSK", "LTC", "LTO", "LUNA", "LUNC", "MAGIC", "MANA", "MANTA", "MASK",
        "MAV", "MAVIA", "MAX", "MBL", "MBOX", "MDT", "MEME", "MERL", "METIS", "MEW",
        "MINA", "MKR", "MLN", "MNT", "MOVR", "MTL", "MYRO", "NEAR", "NEO", "NEXO",
        "NFP", "NKN", "NMR", "NOT", "NTRN", "NULS", "OAX", "OG", "OGN", "OM", "OMG",
        "OMNI", "ONDO", "ONE", "ONG", "ONT", "OOKI", "OP", "ORBS", "ORDI", "ORN",
        "OSMO", "OXT", "PDA", "PENDLE", "PEOPLE", "PEPE", "PERP", "PHA", "PHB",
        "PIXEL", "POLYX", "POND", "POPCAT", "PORTAL", "PORTO", "POWR", "PRCL", "PROM",
        "PROS", "PSG", "PUNDIX", "PYR", "PYTH", "QI", "QKC", "QNT", "QTUM", "QUICK",
        "RAD", "RARE", "RAY", "RDNT", "REI", "REN", "RENDER", "REQ", "REZ", "RIF",
        "RLC", "RON", "ROSE", "RPL", "RSR", "RUNE", "RVN", "SAFE", "SAGA", "SAND",
        "SANTOS", "SC", "SEI", "SFP", "SHIB", "SKL", "SLF", "SLP", "SNT", "SNX",
        "SOL", "SPELL", "SSV", "STEEM", "STG", "STMX", "STORJ", "STPT", "STRAX",
        "STRK", "STX", "SUI", "SUN", "SUPER", "SUSHI", "SXP", "SYN", "SYS", "T",
        "TAO", "TFUEL", "THETA", "TIA", "TKO", "TLM", "TNSR", "TOKEN", "TON", "TRB",
        "TROY", "TRU", "TRX", "TURBO", "TWT", "UFT", "ULTI", "UMA", "UNFI", "UNI",
        "USD", "USDT", "UTK", "UXLINK", "VANRY", "VENOM", "VET", "VIB", "VIC", "VIDT",
        "VITE", "VOXEL", "VTHO", "W", "WAVES", "WAXP", "WEN", "WIF", "WING", "WLD",
        "WOO", "WRX", "XAI", "XCH", "XEC", "XEM", "XLM", "XMR", "XNO", "XRP", "XTZ",
        "XVG", "XVS", "YFI", "YGG", "ZEC", "ZEN", "ZENT", "ZETA", "ZERO", "ZEUS",
        "ZIL", "ZK", "ZKJ", "ZRO", "ZRX"
    ]

    tamano_lote = 10
    total_lotes = (len(lista_simbolos_completa) + tamano_lote - 1) // tamano_lote

    todos_los_dataframes_finales = []
    todos_los_fallos_finales = []

    for i in range(0, len(lista_simbolos_completa), tamano_lote):
        lote_actual = lista_simbolos_completa[i: i + tamano_lote]
        numero_lote_actual = (i // tamano_lote) + 1

        print(f"\n===== Lote {numero_lote_actual}/{total_lotes} ({', '.join(lote_actual)}) =====")

        dataframes_lote, fallos_lote = await funcion_obtener_procesar_lote(lote_actual, inicio_dia_seleccionado, fin_dia_seleccionado)

        todos_los_dataframes_finales.extend(dataframes_lote)
        todos_los_fallos_finales.extend(fallos_lote)

        tiempo_espera = 3
        print(f"===== Lote {numero_lote_actual} completado. Esperando {tiempo_espera} segundos... =====")
        await asyncio.sleep(tiempo_espera)

    funcion_guardar_datos(todos_los_dataframes_finales, inicio_dia_seleccionado)
    funcion_guardar_informe_fallos(todos_los_fallos_finales, inicio_dia_seleccionado)

    print("\n<<<<< Proceso completado >>>>>")

    fecha_archivo = inicio_dia_seleccionado.strftime('%Y%m%d')
    ruta_datos = os.path.join(DIRECTORIO_SALIDA, f'datos_binance_minuto_{fecha_archivo}.csv')
    ruta_fallos = os.path.join(DIRECTORIO_SALIDA, f'informe_fallos_{fecha_archivo}.csv')
    return ruta_datos, ruta_fallos

In [12]:
ruta_archivo_datos, ruta_archivo_fallos = await proceso_principal()

Fecha: 2025-03-06

===== Lote 1/41 (1INCH, AAVE, ACA, ACE, ACH, ACM, ADA, ADX, AERGO, AEVO) =====
  Obteniendo primera mitad para 1INCH...
  Obteniendo primera mitad para AAVE...
  Obteniendo primera mitad para ACA...
  Obteniendo primera mitad para ACE...
  Obteniendo primera mitad para ACH...
  Primera mitad 1INCH: 720 minutos
  Obteniendo primera mitad para ACM...
  Primera mitad ACA: 720 minutos
  Obteniendo primera mitad para ADA...
  Primera mitad ACE: 720 minutos
  Obteniendo primera mitad para ADX...
  Primera mitad AAVE: 720 minutos
  Primera mitad ACM: 720 minutos
  Obteniendo primera mitad para AERGO...
  Primera mitad ACH: 720 minutos
  Obteniendo primera mitad para AEVO...
  Primera mitad ADX: 720 minutos
  Primera mitad AERGO: 720 minutos
  Primera mitad ADA: 720 minutos
  Primera mitad AEVO: 720 minutos
  Obteniendo segunda mitad para 1INCH...
  Obteniendo segunda mitad para ACE...
  Obteniendo segunda mitad para ACA...
  Obteniendo segunda mitad para AAVE...
  Segunda m

In [13]:
if os.path.exists(ruta_archivo_fallos):
    try:
        df_fallos = pd.read_csv(ruta_archivo_fallos)
        print("\nInforme de Fallos:")
        display(df_fallos)
    except Exception:
        print("No se pudo leer el informe de fallos.")
else:
    print(f"\nInforme de fallos no encontrado: {ruta_archivo_fallos}")


Informe de Fallos:


,simbolo,fecha,estado,minutos_obtenidos
0,AIDOGE,2025-03-06,"Error API 400 (1/2): {""code"":-1121,""msg"":""Inva...",0
1,ATH,2025-03-06,"Error API 400 (1/2): {""code"":-1121,""msg"":""Inva...",0
2,BABYDOGE,2025-03-06,"Error API 400 (1/2): {""code"":-1121,""msg"":""Inva...",0
3,BIGTIME,2025-03-06,"Error API 400 (1/2): {""code"":-1121,""msg"":""Inva...",0
4,BLOCK,2025-03-06,"Error API 400 (1/2): {""code"":-1121,""msg"":""Inva...",0
...,...,...,...,...
62,VITE,2025-03-06,Sin datos,0
63,WAVES,2025-03-06,Sin datos,0
64,WRX,2025-03-06,Sin datos,0
65,XEM,2025-03-06,Sin datos,0


In [14]:
if os.path.exists(ruta_archivo_datos):
    try:
        df_datos_muestra = pd.read_csv(ruta_archivo_datos, nrows=10)
        print(f"\nMuestra del archivo de datos ({os.path.basename(ruta_archivo_datos)}):")
        display(df_datos_muestra)
    except Exception:
        print("No se pudo leer el archivo de datos.")
else:
    print(f"\nArchivo de datos no encontrado: {ruta_archivo_datos}")


Muestra del archivo de datos (datos_binance_minuto_20250306.csv):


,timestamp,symbol,open,high,low,close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume
0,2025-03-06 00:00:00+0000,1INCH,0.23400,0.23410,0.23390,0.23410,8029.000,1878.889200,9,5689.000,1331.563200
1,2025-03-06 00:00:00+0000,AAVE,220.91000,221.00000,220.61000,220.72000,79.552,17568.970720,211,31.828,7030.127490
2,2025-03-06 00:00:00+0000,ACA,0.04270,0.04270,0.04260,0.04270,996.570,42.517375,4,634.930,27.111511
3,2025-03-06 00:00:00+0000,ACE,0.79400,0.79600,0.79400,0.79600,8102.800,6437.004300,25,8102.800,6437.004300
4,2025-03-06 00:00:00+0000,ACH,0.02634,0.02634,0.02633,0.02633,12601.000,331.875830,9,5378.000,141.656520
5,2025-03-06 00:00:00+0000,ACM,0.96700,0.96800,0.96600,0.96800,287.900,278.419600,15,153.000,148.074700
6,2025-03-06 00:00:00+0000,ADA,0.97410,0.97580,0.97230,0.97440,668067.600,650840.583750,1798,277734.500,270686.237670
7,2025-03-06 00:00:00+0000,ADX,0.11800,0.11810,0.11790,0.11810,7395.000,872.329600,12,1323.000,156.146900
8,2025-03-06 00:00:00+0000,AERGO,0.07420,0.07420,0.07420,0.07420,7800.000,578.760000,4,7800.000,578.760000
9,2025-03-06 00:00:00+0000,AEVO,0.13680,0.13680,0.13670,0.13680,17015.220,2326.153446,13,4565.690,624.239475
